**Building a graph based on a road network**

In the previous sections, we learned about the fundamentals of the graph and many of the operations that we can perform on a graph using sample data that we defined. In this section, we will discuss how to import real-world data such as a road network and translate it into a graph and perform analyses on that. For this purpose, we will be importing sample Open Street Maps (OSM) road data that we have provided with this chapter. 

***Open Street Maps data***

OSM provides a rich set of open source and crowdsourced geographic data of many parts of the world. In particular, we are interested in the road data they provide. For the US, they are baselined on census TIGER road shapefiles and benefit from continuous updates by the active OSM community. OSM road data provide critical information such as the following:

- Road geometry
- Road direction
- Road speed limit
- Traffic signs and impediments (pedestrian crossings and so on)

Exploring the road data 
For reading the shapefiles, we will be using the GeoPandas library and Shapely geometry modules. If GeoPandas is not installed, install it using pip or any similar package manager. The pip installation code is shown as follows

In [ ]:
!pip install geopandas

To execute the code further, please download the BayArea4 shapefiles from our data repository into the folder where your Jupyter Notebook resides. 

Let's now import the rest of the libraries required to create our graph:

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import pandas as pd

In [ ]:
# Load the data

roads_df = gpd.read_file('Roads_BayArea4.shp')

In [ ]:
# Read the data

roads_df.tail()

***DataFrame view of roads GeoDataFrame***

The maxspeed column is of particular interest since it defines the maximum speed in which someone can traverse that road segment as permitted by law. If we need to create a time-based routing, we need to leverage this field. But before doing that, we need to make sure that the field provides us with complete information. To do that, let's execute the following code, which provides us with the sum of rows with null values in each column:

In [ ]:
roads_df.isnull().sum(axis=0)

A first look at the preceding response looks promising. None of the rows in the maxspeed column have null values. However, it might be possible that most of the null values for the maxspeed column are substituted by 0, which is an unacceptable value if we were to use the maxspeed column to derive the time taken to traverse a road segment. Let's confirm that this isn't the case:

In [ ]:
len(roads_df[roads_df["maxspeed"] == 0])

35,000 out of 54,000 records have 0 as a value for maxspeed. So, we need to find a proxy to calculate the values for maxspeed. This will be discussed in detail in the following sections. For now, let's try to visualize the roads GeoDataframe as a simple plot using the following lines of code:

In [ ]:
%matplotlib inline
roads_df.plot(figsize=(10,10))

The simple plot of the roads' GeoDataFrame shown here reveals to us the line geometry of the roads and the network it forms:

**Creating a graph from a DataFrame **

The following steps are required to create a graph from our DataFrame:

- Reading and exploding the geometry
- Calculating the distance of each edge in meters
- Finding a proxy for zero values of maxspeed
- Accounting for directionality
- Calculating drive time in seconds for each edge 

***Reading and exploding the geometry***

The geometry column holds the key to constructing the edges required for our road network graph. Let's read just a random row for and play around with the geometry column and try to make sense of it:



In [ ]:
frow = roads_df.iloc[1000]
print("{frow}\n\n")
print("Coords : {frow['geometry'].xy}\n\n")
print("Length {frow['geometry'].length}")

We picked up the 1,001st row and tried to print out the information in the row and the geometric information such as the coordinates in the geometry LineString and the length of the LineString. The length of the LineString is in decimal degrees and doesn't make much sense and is not useful to us, but the coordinates of the LineString are. The coordinates are available as a tuple of NumPy arrays, with the first item in the tuple being the NumPy array of longitudes and the second item being the array of latitudes:

How do these coordinates look in the real world? Well, wonder no more; a simple matplot plotting will answer that:

In [ ]:
x, y = frow['geometry'].xy
fig = plt.figure(1, figsize=(5,5), dpi=90)
ax = fig.add_subplot(111)
ax.plot(x, y)

Twenty-six coordinates were to create this sinuous road segment. This means that 25 graph edges be created from the 26 coordinates

The following lines of codes accomplish these requirements.:

In [ ]:
def split_data_frame_list(df, target_column, col_name, output_type=list):
    ''' 
    Accepts a column with multiple types and splits list variables to several rows.
    df: dataframe to split
    target_column: the column containing the values to split
    col_name : name of the split column
    output_type: type of column to be split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''

    row_accumulator = []

    def split_list_to_rows(row):
        split_row = row[target_column]
        if isinstance(split_row, list):
            for s in split_row:
                new_row = row.to_dict()
                new_row[col_name] = output_type(s)
                row_accumulator.append(new_row)
        else:
            new_row = row.to_dict()
            new_row[col_name] = output_type(split_row)
            row_accumulator.append(new_row)

    df.apply(split_list_to_rows, axis=1)
    new_df = pd.DataFrame.from_records(row_accumulator, columns=[df.columns])
 return new_df

The preceding function explodes the road segments into individual rows, so that we have one edge per row. After defining the preceding function, we can execute the code provided as follows. This will create a new DataFrame: edge_df. edge_df will contain a geometry column that contains the edge coordinates. Two separate columns, source and target, should also be created. The source column will contain the first coordinate of the edge, and the target column will contain the second coordinate of the edge:

In [ ]:
roads_df["geometry_arr"] = roads_df["geometry"].apply(lambda geom : list(zip(*geom.xy)))
roads_df["edge_list"] =  roads_df["geometry_arr"].apply(lambda coords : [[coords[i], coords[i+1]] for i in range(len(coords) - 1)])

edge_df = split_data_frame_list(roads_df, "edge_list", "geometry")
edge_df["source"] = edge_df["geometry"].apply(lambda edge: edge[0])
edge_df["target"] = edge_df["geometry"].apply(lambda edge: edge[1])
edge_df = edge_df.drop('geometry', axis = 1)
edge_df = edge_df.drop('geometry_arr', axis = 1)
roads_df = roads_df.drop(["geometry_arr", "edge_list"], axis = 1)
edge_df.head()

Calculating the distance of edges
We can use the haversine function discussed in earlier chapters to calculate the distance for each edge. The following lines of code will add a dist column to edge_df and populate it with the length of the edge in meters:

In [ ]:
from math import radians, cos, sin, asin, sqrt, atan2

def haversine(loc1, loc2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1 = loc1
    lon2, lat2 = loc2
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6378100 # Radius of earth in meters. Use 3956 for miles
    return c * r


edge_df["dist"] = edge_df.apply(lambda row: haversine(row["source"], row["target"]), axis = 1)


***Finding a proxy for maximum speed ***

Our job, at this point, is to find a proxy for maximum speed values when the value is not present (or populated by zero). The fclass column provides the best opportunity to impute this data. The data we are working with corresponds to the state of California. The following speed lookup for the fclass type holds good enough for California:

In [ ]:
speed_limits_in_mph = {
  "living_street"     : 10,
  "motorway"          : 65, 
  "motorway_link"     : 40, 
  "primary"           : 55,
  "primary_link"      : 30,
  "residential"       : 25, 
  "secondary"         : 35,
  "secondary_link"    : 25,
  "service"           : 15, 
  "tertiary"          : 35,
  "tertiary_link"     : 25,
  "trunk"             : 55, 
  "trunk_link"        : 30,
  "unclassified"      : 55, 
  "unknown"           : 40
}

Update the maxspeed column in edge_df with the preceding lookup dictionary. Remember, the maxspeed column is in kilometers per hour and the lookup table speeds are provided in miles per hour. So, while updating maxspeed, a multiplicative factor of 1.609 is added to the lookup table speeds (since 1 mile = 1.609 km):

In [ ]:
edge_df["maxspeed"] = edge_df.apply(lambda row: speed_limits_in_mph[row["fclass"]] * 1.609 if row["maxspeed"] == 0 else row["maxspeed"], axis = 1)

***Accounting for directionality***

The directionality of the edges is defined in the oneway column. The definition for the column is provided in the data dictionary provided by the Geofabrik download site. 

For edges with oneway == 'F', no changes are required. For edges with oneway == 'T', the current edges have to flip, that is, the source and target values must be swapped. For edges with oneway == 'B', additional edges replicating the edge need to be added and then swapped. The following lines of code will achieve the preceding objective

In [ ]:
pd.options.mode.chained_assignment = None
nt = edge_df.query("oneway != 'T'", inplace = False)
b = edge_df.query("oneway == 'B'", inplace = False)

src_tmp = b["source"]
b["source"] = b["target"]
b["target"] = src_tmp

t = edge_df.query("oneway == 'T'" , inplace = False)
src_tmp = t["source"]
t["source"] = t["target"]
t["target"] = src_tmppd.options.mode.chained_assignment = None
nt = edge_df.query("oneway != 'T'", inplace = False)
b = edge_df.query("oneway == 'B'", inplace = False)

src_tmp = b["source"]
b["source"] = b["target"]
b["target"] = src_tmp

t = edge_df.query("oneway == 'T'" , inplace = False)
src_tmp = t["source"]
t["source"] = t["target"]
t["target"] = src_tmp

edge_df = pd.concat([nt, b, t], ignore_index=True)

***Calculating drivetime***

Drivetime for the edges can be calculated using the middle school Time and Distance calculations. The following formulae and conversion will help our time calculations: 

Time = Distance / SpeedTime (in seconds) = (Distance (in meters) / Speed (in kmph) ) * (18/5)

Let's implement this formula with the following line of code

In [ ]:
edge_df["time"] = (edge_df["dist"]/edge_df["maxspeed"]) * (18/5)

We have hence effectively added a time column. No more processing is needed for the DataFrame at this point. We are ready to convert this into a graph:

***Building the graph***

We have already discussed how to convert a pandas DataFrame into a graph using the networkx method known as from_pandas_edgelist(). Since our edge_df already contains the source and target columns, we need not mention it explicitly in the method. To keep the footprint of the graph light, let's only include the most essential data from the DataFrame as edge attributes. Here, we are adding osm_id, name, and time as edge attributes:

In [ ]:
G = nx.from_pandas_edgelist (df = edge_df, edge_attr = ["osm_id", "name", "time"], create_using  = nx.DiGraph())

With that preceding line of code, we have built a graph from a road network GeoDataframe. In the next section, we will be doing all of the exciting stuff such as performing analyses with this graph

***Shortest path analyses on the road network graph***

In this section, we will be performing some of the analyses that we did earlier on the new graph, such as the following:

- Dijkstra's shortest path analysis
- Dijkstra's shortest path cost
- Single-source Dijkstra's path length

***Dijkstra's shortest path analysis***

We have waited so long to perform the shortest path analysis on our graph. So, without further ado, let's run the shortest path algorithm on our graph, optimizing for time. To run the shortest path algorithm, we need to know the source node ID and target node ID. Here, our node IDs are coordinates. Right now, let's pick the node IDs randomly:

In [ ]:
nodelist = list(G.nodes())
import random
random.seed(0)

The following lines of code select two nodes at random and assign them as the source and the target node, respectively. This is used as input to the dijkstra_path() function. As we have seen earlier, this function returns a list of node IDs in order, joining the ones that will make up the shortest path from the source node to the target node. Since the returned node IDs are already coordinates, we can create a Shapely LineString from it and plot it on the roads GeoDataFrame

In [ ]:
# Selecting random source and target
src, target = nodelist[random.randint(0, len(nodelist)-1)], nodelist[random.randint(0, len(nodelist)-1)]

#Calculating Shortest path
sp = nx.dijkstra_path(G, src, target, weight='time')

#Creating Shortest Path geometry with the result
spd = gpd.GeoDataFrame({"geometry" : gpd.GeoSeries(LineString([(item) for item in sp]))})

#Plot the geometry
ax = roads_df.plot(color='gray', figsize = (12, 12), linewidth = 0.7, alpha = 0.5)
spd.plot(ax=ax, color='black', linewidth=2)

ax.text(src[0], src[1], s = 'Source', fontsize=12 ,color='r')
ax.text(target[0], target[1], s = 'Target', fontsize=12, color='r')
ax.scatter([src[0], target[0]],[src[1], target[1]], marker = '^', s = 100) #Markers

ax.axis('equal')
ax.axis('off')

This code generates a plot as follows. The dark line represents the shortest path between a source and target node in the road graph (represented by gray lines):

***Dijkstra's shortest path cost***

To get the time cost of travel from the source node to the target node, just use the following one-liner:

In [ ]:
nx.dijkstra_path_length(G, src, target, weight='time')//60

***Single source Dijkstra's shortest path cost***

When we need to know the cost to reach all possible nodes from the source node, with a maximum accumulated cost of 30 minutes (1800 seconds), we can use the single_source_dijkstra_path_length() method, as follows:

In [ ]:
spl = nx.single_source_dijkstra_path_length(G, src, cutoff = 1800, weight="time")
dict([spl.popitem() for i in range(0, 10)])

The response of the method, stored in the splvariable, is a Python dictionary with the node IDs as the keys of the dictionary item and corresponding time cost (in seconds) to reach the node from the defined source node as dictionary values. The last ten items of the dictionary are displayed, as follows: 

***Concept of isochrones***

Shortest paths are a navigational solution that provides the shortest paths and the shortest time (or distance) from a source to a target node. What if we need a solution with all possible areas that we can reach from a source node within a given time or distance cut-off? This is very important in commute-based searches, which can answer questions such as the following: 

- What are the nearest coffee shops within 15 minutes by car?
- What are the open home listings that are 30 minutes from my office? 
- Can any fire truck stationed at its base location reach my house within 10 minutes? 
- What are the areas that are under-represented by public transit agencies?

The idea we are looking at is known as an isochrone. Etymologically, it means same time (iso = same and chrono = time). More specifically, it means the areas that you can reach within the same time from a source location by a particular mode of transport under normal circumstances:

The preceding graph shows the isochrone as a red, shaded area. The isochrone represents all possible locations that we can drive to from the yellow dot shown, as per our graph calculations. 

***Constructing an isochrone***

As you might have guessed, we can use the response from the single_source_dijkstra_path_length() function to create an isochrone. But once we execute this method, we might have to further construct a polygon from the returned node locations. There are different methods to create a polygon given a mesh of points, but we will be discussing an approach known as the concave hull. Concave hulls are created by repeated triangulation of available points in the most efficient way. The implementation of this triangulation is provided by a scipy.spatial module known as Delaunay. The process of creating an isochrone polygon is as follows: 

1. Create Delauney triangles with the location of the result nodes, as shown here: 

In [ ]:
# Compute Delaunay triangles
tri = Delaunay(coords)

2. Loop through each triangle and calculate the circumradius of the triangle
3. If the circumradius of the triangle is less than a preset value (say, 1/150), add the vertices of the triangle to a set
4. Construct a series of polygons from the set of vertices
5. Merge the polygons


The code implementation of this logic can be found in the code repository under a function named generate_alpha_poly(). Once we have defined the alpha polygon generator function, we can use the response from our single_source_dijkstra_path_length() method, stored in the spl variable. Since we are using the cutoff parameter, we just need the keys of the spl dictionary. For a 30-minute isochrone, we will receive a lot of nodes as coordinates; hence, it is OK to round off the decimal degrees to 3 digits in the node coordinates: 

In [ ]:
pts = np.array(list(spl.keys()))
pts = np.round(pts, 3)
pts = np.unique(pts,axis = 0)
poly = generate_alpha_poly(pts, 150)

When this polygon is plotted, it looks exactly like a plot that was shown at the beginning of this section: the diagram titled Isochrone. 